In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
wego = pd.read_csv("../data/Headway Data, 8-1-2023 to 9-30-2023.csv")

wego.head()

1.What is the overall on-time performance, and what do the overall distributions of adherence and headway deviation look like?






In [ ]:
#Filtering columns
relevant_columns = ['ROUTE_ABBR', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ADHERENCE', 'HDWY_DEV','SCHEDULED_HDWY']
filtered_data = wego.loc[:, relevant_columns]


In [ ]:
ontime_percentage = (wego['ADJUSTED_ONTIME_COUNT'].value_counts(normalize=True) * 100).round(2)

# '%' added
ontime_percentage_formatted = ontime_percentage.map("{:.2f}%".format)

# Results
print("Percentage of ADJUSTED_ONTIME_COUNT values:")
print(ontime_percentage_formatted)

In [ ]:
#Uncleaned 
wego['HDWY_DEV_PCT'] = wego['HDWY_DEV'] / wego['SCHEDULED_HDWY']

In [ ]:
#Cleaned PCT without inf and NaN's
wego['HDWY_DEV_PCT_C'] = wego['HDWY_DEV'] / wego['SCHEDULED_HDWY']

# Replace inf values with NaN
wego['HDWY_DEV_PCT_C'] = wego['HDWY_DEV_PCT_C'].replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN values
wego_cleaned = wego.dropna(subset=['HDWY_DEV_PCT_C'])

In [ ]:
#Plots
plt.figure(figsize=(12, 6))

# Distribution of adherence
plt.subplot(1, 2, 1)
sns.histplot(filtered_data['ADHERENCE'], kde=True, color='lightpink')
plt.title('Distribution of Adherence')
plt.xlabel('Adherence (minutes)')
plt.ylabel('Frequency')


# Distribution of headway deviation
plt.subplot(1, 2, 2)
sns.histplot(filtered_data['HDWY_DEV'], kde=True, color='lightpink')
plt.title('Distribution of Headway Deviation')
plt.xlabel('Headway Deviation (minutes)')
plt.ylabel('Frequency')


plt.tight_layout()
plt.show()

In [ ]:
#Histograms with pretty lines
plt.figure(figsize=(12, 6))

#'ADHERENCE'
plt.subplot(1, 2, 1)
sns.histplot(filtered_data['ADHERENCE'], kde=True, color='lightpink')
plt.title('Histogram of Adherence')
plt.xlabel('Adherence (minutes)')
plt.ylabel('Frequency')
plt.axvline(x=-6, color='purple', linestyle='--', label='Late Threshold (-6 min)')  # Adding a line indicating late threshold
plt.axvline(x=1, color='green', linestyle='--', label='Early Threshold (1 min)')  # Addin a line indicating early threshold
plt.xlim(left=-30, right=30)
plt.legend()

#'HDWY_DEV' 
filtered_data['HDWY_DEV_PCT_C'] = filtered_data['HDWY_DEV'] / filtered_data['SCHEDULED_HDWY']
plt.subplot(1, 2, 2)
sns.histplot(filtered_data['HDWY_DEV_PCT_C'], kde=True, color='#6F4B9E')
plt.title('Headway Deviation Percent')
plt.xlabel('Headway Deviation Precent')
plt.ylabel('Frequency')
plt.xlim(left=-2, right=2)  


plt.tight_layout()
plt.show()

2. How does direction of travel, route, or location affect the headway and on-time performance?

In [ ]:
grouped_abbr = wego.groupby('ROUTE_ABBR')

In [ ]:
grouped_route = wego.groupby('ROUTE_DIRECTION_NAME')

In [ ]:
statistics = grouped_abbr.agg({'HDWY_DEV_PCT_C': ['mean', 'std'], 'ADJUSTED_ONTIME_COUNT': 'mean'})

In [ ]:
statistics

In [ ]:
# Group by 'ROUTE_DIRECTION_NAME' & 'ROUTE_ABBR'
grouped_route = wego.groupby(['ROUTE_ABBR', 'ROUTE_DIRECTION_NAME'])

#Stats
statistics = grouped_route.agg({'HDWY_DEV_PCT_C': ['mean', 'std'], 'ADJUSTED_ONTIME_COUNT': 'mean'})

In [ ]:
statistics